In [1]:
import numpy as np
import cvxopt

$$
\begin{equation*}
\begin{aligned}
\min_{x} \quad & \frac{1}{2} x^2 + 6 \\
\text{s.t.} \quad  & x \geq 0 \\
                   & x \leq 5 \\
\end{aligned}
\end{equation*}
$$

Who knows the answer without jumping through hoops?

### Lagrange Multipliers

$$
\begin{equation*}
\begin{aligned}
\min_{x} \quad & \frac{1}{2} x^2 + 6 \\
\text{s.t.} \quad  & x \geq 0 \\
                   & x \leq 5 \\
\end{aligned}
\end{equation*}
$$

What's the dual to this? (On the board)

$$
\sup_{\lambda_1,\lambda_2\geq 0}\tfrac{1}{2} (\lambda_1)^2 - \lambda_1\lambda_2 + \tfrac{1}{2}(\lambda_2)^2-5\lambda_2+6
$$

### CVXOPT

One objective, two constraints:

$$
\begin{array}{ccc}
\begin{aligned}
\min_{x} \quad & \frac{1}{2} x^2 + 6 \\
\text{s.t.} \quad  & x \geq 0 \\
                   & x \leq 5 \\
\end{aligned}
&
\qquad \rightarrow \qquad
&
\begin{aligned}
\min_{\theta} \quad & \tfrac{1}{2}\theta^\top P \theta + q^\top \theta + r \\
\text{s.t.}\quad & G\theta \preceq h \\
& A\theta = b
\end{aligned}
\end{array}
$$

Rewrite the constraints

$$
-x \leq 0\\
x  \leq 5
$$

Assemble the matrices

$$
\begin{array}{cc}
P = I = [1]
&
q = [0]
\\
\quad
\\
G = \begin{pmatrix}-1 \\ 1\end{pmatrix}
&
h = \begin{pmatrix} 0 \\ 5\end{pmatrix}
\end{array}
$$

In [5]:
def do_primal():
  # Convert from equations of hyperplane and point to standard form
  P = np.array([1])  # Objective quadratic terms
  q = np.array([0])  # Objective linear terms (as a COLUMN vector)
  G = np.array([[-1], [1]])                   # Equality constraint
  h = np.array([[ 0], [5]])        # Equality constraint

  # Rearrange for use with CVXOPT
  P, q, G, h = [np.atleast_2d(x) for x in [P, q, G, h]]
  P, q, G, h = [x.astype(float)  for x in [P, q, G, h]]  
  P, q, G, h = [cvxopt.matrix(M) for M in [P, q, G, h]]

  # Actually use CVXOPT
  sol = cvxopt.solvers.qp(P, q, G=G, h=h)
  x = np.array(sol['x']).flatten()
  return x

do_primal()

     pcost       dcost       gap    pres   dres
 0:  1.3889e+00 -6.3889e+00  8e+00  0e+00  9e-16
 1:  3.7193e-01 -4.2193e-01  8e-01  3e-16  3e-16
 2:  5.4283e-02 -6.4482e-02  1e-01  4e-17  1e-16
 3:  7.8363e-03 -8.7057e-03  2e-02  4e-16  0e+00
 4:  1.1254e-03 -1.2449e-03  2e-03  1e-16  7e-18
 5:  1.6138e-04 -1.7774e-04  3e-04  3e-18  0e+00
 6:  2.3132e-05 -2.5444e-05  5e-05  3e-16  1e-17
 7:  3.3152e-06 -3.6445e-06  7e-06  4e-16  4e-19
 8:  4.7507e-07 -5.2217e-07  1e-06  5e-17  4e-19
 9:  6.8078e-08 -7.4820e-08  1e-07  1e-16  2e-19
10:  9.7554e-09 -1.0721e-08  2e-08  5e-17  3e-20
Optimal solution found.


array([0.00013968])

What would it look like to use CVXOPT to solve the dual to this problem?

$$
\sup_{\lambda_1,\lambda_2\geq 0} -\tfrac{1}{2} (\lambda_1)^2 + \lambda_1\lambda_2 - \tfrac{1}{2}(\lambda_2)^2-5\lambda_2+6
$$

One objective, two constraints:

$$
\begin{array}{ccc}
\begin{aligned}
\sup_{\lambda_1, \lambda_2} \quad & -\tfrac{1}{2} (\lambda_1)^2 + \lambda_1\lambda_2 - \tfrac{1}{2}(\lambda_2)^2-5\lambda_2+6 \\
\text{s.t.} \quad  & \lambda_1 \geq 0 \\
                   & \lambda_2 \geq 0 \\
\end{aligned}
&
\qquad \rightarrow \qquad
&
\begin{aligned}
\min_{\theta} \quad & \tfrac{1}{2}\theta^\top P \theta + q^\top \theta + r \\
\text{s.t.}\quad & G\theta \preceq h \\
& A\theta = b
\end{aligned}
\end{array}
$$

Rewrite the objective and constraints:

$$
\begin{aligned}
\inf_{\lambda_1, \lambda_2} \quad & \tfrac{1}{2} (\lambda_1)^2 - \lambda_1\lambda_2 + \tfrac{1}{2}(\lambda_2)^2 + 5\lambda_2 - 6 \\
\text{s.t.} \quad  & - \lambda_1 \leq 0 \\
                   & - \lambda_2 \leq 0 \\
\end{aligned}
$$

Assemble the matrices

$$
\begin{array}{cc}
P = \begin{pmatrix}  1 & -1 \\ -1 &  1 \end{pmatrix}
&
q = \begin{pmatrix}  0      \\  5      \end{pmatrix}
\\
\quad
\\
G = \begin{pmatrix} -1 & 0 \\ 0 & -1 \end{pmatrix}
&
h = \begin{pmatrix}  0 \\  0 \end{pmatrix}
\end{array}
$$

In [8]:
def do_dual():
  # Convert from equations of hyperplane and point to standard form
  P = np.array([[ 1, -1], [-1,  1]])  # Objective quadratic terms
  q = np.array([[ 0]    , [ 5]    ])  # Objective linear terms (as a COLUMN vector)
  G = np.array([[-1, 0], [0, -1]])                   # Equality constraint
  h = np.array([[ 0], [ 0]])        # Equality constraint

  # Rearrange for use with CVXOPT
  P, q, G, h = [np.atleast_2d(x) for x in [P, q, G, h]]
  P, q, G, h = [x.astype(float)  for x in [P, q, G, h]]  
  P, q, G, h = [cvxopt.matrix(M) for M in [P, q, G, h]]

  # Actually use CVXOPT
  sol = cvxopt.solvers.qp(P, q, G=G, h=h)
  x = np.array(sol['x']).flatten()
  return x

do_dual()

     pcost       dcost       gap    pres   dres
 0: -1.5278e+01 -1.3889e+00  8e+00  6e+00  4e-16
 1: -7.4301e-01 -3.7193e-01  8e-01  3e-01  5e-16
 2:  6.4482e-02 -5.4283e-02  1e-01  1e-16  3e-15
 3:  8.7057e-03 -7.8363e-03  2e-02  3e-17  2e-16
 4:  1.2449e-03 -1.1254e-03  2e-03  1e-20  2e-16
 5:  1.7774e-04 -1.6138e-04  3e-04  3e-21  2e-16
 6:  2.5444e-05 -2.3132e-05  5e-05  5e-22  5e-16
 7:  3.6445e-06 -3.3152e-06  7e-06  1e-23  2e-16
 8:  5.2217e-07 -4.7507e-07  1e-06  3e-19  2e-16
 9:  7.4820e-08 -6.8078e-08  1e-07  5e-20  4e-16
10:  1.0721e-08 -9.7554e-09  2e-08  3e-20  4e-16
Optimal solution found.


array([1.39681307e-04, 1.93186343e-10])

Look at the outputs of the two runs of CVXOPT. Notice anything interesting?

### Bonus: Visualization

[Visualization in Desmos](https://www.desmos.com/3d/1zgmx7saep). This helped me understand what's going on, ***after*** working through the problem. I recommend fighting the problem a bit first.

# Your problem

I've repeated this from Recitation 2's notebook. It's worth doing this a couple times. I'm not doing it here or in recitation.

$$
\begin{equation*}
\begin{aligned}
\min_{x} \quad & \frac{1}{2} (x-1)^2 \\
\text{s.t.} \quad  & x \geq 2 \\
\end{aligned}
\end{equation*}
$$